In [ ]:
import jax.random as jrand

import genjax
from genjax import gen, pjax_to_jax


@gen
def model(idx, mu):
    return genjax.normal.switch(genjax.normal)(idx, (mu, 2.0), (mu, 1.0)) @ "x"


pjax_to_jax(jrand.key(5), model.simulate)((0, 0.3))